### Using a Pipeline

Now that you have gone through the basic steps of running the IceNet model you may wish to establish a framework to run the model automatically. This is often called a Pipeline. A Pipeline can schedule ongoing model runs or run multiple model variations simulataneously.

![Diagram of Icenet and it's pipeline](./pipeline_diagram3.png "Icenet pipeline diagram displaying process blocks and data being processed from input on the left to output on the right, through the pipeline")

To automatically produce daily IceNet forecasts we train multiple variations of the model, each with slightly differen't starting conditions. We call this ensemble training. Then we run predictions for each model variation, producing a mean and error across the whole model ensemble.

### Ensemble Running

To do this, an icenet-pipeline repository is available. The icenet-pipeline offers the `run_predict_ensemble` script which operates similarly to the above model training script.

### Get the IceNet Pipeline

Before progressing you will need clone the icenet-pipeline repository:

```bash
git clone https://www.github.com/icenet-ai/icenet-pipeline.git ../green
ln -s ../green ../notebook-pipeline
cd ../notebook-pipeline
```

We clone a 'fresh' pipeline repository into a folder called 'green' (as an arbitrary way of identifying the fresh pipeline) and then symbolically link to it. This allows us to symbolically swap to another pipeline later if we want to.

In [1]:
import os
!find .. -maxdepth 1 -type l -ls

20566646526    0 lrwxrwxrwx   1 matsco   ame_eng         5 Jan 22 14:47 ../notebook-pipeline -> green


### Configure the Pipeline

The `run_predict_ensemble` script will submit jobs to the HPC and those jobs are generated using `.yaml` templates.  
There are `.yaml` templates in the `ensemble/` folder of the cloned pipeline repository which are used to configure the jobs. In particular [`predict.tmpl.yaml`](https://github.com/icenet-ai/icenet-pipeline/blob/main/ensemble/predict.tmpl.yaml) for the training ensemble jobs.

Move into the `notebook-pipeline` directory.

In [2]:
os.chdir("../notebook-pipeline")
!pwd

/data/hpcdata/users/matsco/icenet0.2.7/green


Before running the Predict Ensemble you must check that pipeline `ENVS` configuration is correct.

In [3]:
!find . -maxdepth 1 -type l -ls

22882332080    0 lrwxrwxrwx   1 matsco   ame_eng        12 Jan 22 14:47 ./ENVS -> ENVS.example


You can see that the `ENVS` config points to `ENVS.example`. Within this file, make sure the second and third 'export' commands refer to the correct location of your cloned icenet repository and your conda environment respectively.
><pre>
export ICENET_HOME=${ICENET_HOME:-${HOME}/icenet/${ICENET_ENVIRONMENT}}
export ICENET_CONDA=${ICENET_CONDA:-${HOME}/conda-envs/icenet}
</pre>

### Run the Pipeline
First look at the required input arguments for running the prediction ensemble.

In [4]:
!../icenet/run_predict_ensemble.sh --help

Usage ../icenet/run_predict_ensemble.sh NETWORK DATASET NAME DATEFILE [LOADER]


So to run the ensemble we use:  

| argument               |description |
|-----------------------:|:------------|
|*neural network name*   | - notebook_ensemble  
|*dataset*               | - notebook_data  
|*name of ensemble run*  | - example_south_ensemble_forcast  
|*which dates to run*    | - testdates.csv  


***... and now there are more arguments/switches that aren't documented in the above help***

In [22]:
!../icenet/run_predict_ensemble.sh \
    -b 1 -f 0.6 -p bashpc.sh \
    notebook_ensemble notebook_data example_south_ensemble_forecast testdates.csv

ARGS: -b 1 -f 0.6 -p bashpc.sh notebook_ensemble notebook_data example_south_ensemble_forecast testdates.csv
ARGS = -x arg_batch=1 arg_filter_factor=0.6 arg_prep=bashpc.sh , Leftovers: notebook_ensemble notebook_data example_south_ensemble_forecast testdates.csv
ln: failed to create symbolic link ‘ensemble/example_south_ensemble_forecast/predict_dates.csv’: File exists
Running model_ensemble ./tmp.gsBgXiFyHv.predict slurm -x arg_batch=1 arg_filter_factor=0.6 arg_prep=bashpc.sh 
[01-02-24 08:58:11    :INFO    ] - Model Ensemble Runner
[01-02-24 08:58:11    :INFO    ] - Validated configuration file ./tmp.gsBgXiFyHv.predict successfully
[01-02-24 08:58:11    :INFO    ] - Importing model_ensembler.cluster.slurm
[01-02-24 08:58:11    :INFO    ] - Running batcher
[01-02-24 08:58:11    :INFO    ] - Start batch: 2024-02-01 08:58:11.338585
[01-02-24 08:58:11    :INFO    ] - Running cycle 1
[01-02-24 08:58:11    :INFO    ] - Running command: /usr/bin/ln -s ../../data
[01-02-24 08:58:11    :WARNI

***Here I want to highlight what the outcome of the ensemble was and show something graphical.***

As with the previous example, the individual numpy outputs, samples and sample weights are deposited into `/results/predict` for each ensemble member. However, the ensemble also runs `icenet_output` to generate __a CF-compliant NetCDF containing the forecasts requested__ which can then be post-processed or [deposited to an external location](#Uploading-to-Azure) (which is the platform for the [wider IceNet forecasting infrastructure](https://github.com/alan-turing-institute/IceNet-Project)). 

**Move back into the `icenet-notebook` directory before generating the pipeline output.**

In [5]:
os.chdir("../icenet-notebooks-review")
!pwd

/data/hpcdata/users/matsco/icenet0.2.7/icenet-notebooks-review


In [6]:
!icenet_output -o results/predict example_south_forecast notebook_data testdates.csv

2024-02-19 08:16:01.669961: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-19 08:16:01.723357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-19 08:16:30.674715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[19-02-24 08:17:19 :INFO    ] - Loading configuration ./dataset_config.notebook_data.json
[19-02-24 08:17:19 :INFO    ] - Training dataset path: ./network_datasets/notebook_data/south/train
[19-02-24 08:17:19 :INFO    ] - Validation dataset path: ./n

Note that the ensemble run automatically handles this generation of output for the ensemble. For a single run this is relatively meaningless as there is only a single model making predictions, giving no uncertainty quantification, __so this is provided as an example only__. _Please review the `-h` help option for the script to gain further insight the options available._


### View the forecast output from the pipeline

In [7]:
from icenet.plotting.video import xarray_to_video as xvid
from icenet.data.sic.mask import Masks
from IPython.display import HTML
import xarray as xr, pandas as pd, datetime as dt

ds = xr.open_dataset("results/predict/example_south_forecast.nc")
land_mask = Masks(south=True, north=False).get_land_mask()
ds.info()

2024-02-19 08:18:45.853115: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-19 08:18:45.907847: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-19 08:18:49.194262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


xarray.Dataset {
dimensions:
	time = 2 ;
	yc = 432 ;
	xc = 432 ;
	leadtime = 7 ;

variables:
	int32 Lambert_Azimuthal_Grid() ;
		Lambert_Azimuthal_Grid:grid_mapping_name = lambert_azimuthal_equal_area ;
		Lambert_Azimuthal_Grid:longitude_of_projection_origin = 0.0 ;
		Lambert_Azimuthal_Grid:latitude_of_projection_origin = -90.0 ;
		Lambert_Azimuthal_Grid:false_easting = 0.0 ;
		Lambert_Azimuthal_Grid:false_northing = 0.0 ;
		Lambert_Azimuthal_Grid:semi_major_axis = 6378137.0 ;
		Lambert_Azimuthal_Grid:inverse_flattening = 298.257223563 ;
		Lambert_Azimuthal_Grid:proj4_string = +proj=laea +lon_0=0 +datum=WGS84 +ellps=WGS84 +lat_0=-90.0 ;
	float32 sic_mean(time, yc, xc, leadtime) ;
		sic_mean:long_name = mean sea ice area fraction across ensemble runs of icenet model ;
		sic_mean:standard_name = sea_ice_area_fraction ;
		sic_mean:short_name = sic ;
		sic_mean:valid_min = 0 ;
		sic_mean:valid_max = 1 ;
		sic_mean:ancillary_variables = sic_stddev ;
		sic_mean:grid_mapping = Lambert_Azimuth

In [8]:
forecast_date = ds.time.values[0]
print(forecast_date)

2020-04-01T00:00:00.000000000


In [9]:
fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
fc['time'] = [pd.to_datetime(forecast_date) \
              + dt.timedelta(days=int(e)) for e in fc.time.values]

anim = xvid(fc, 15, figsize=4, mask=land_mask)
HTML(anim.to_jshtml())

### Uploading to Azure

The following command uploads a specific date from the `icenet_output` produced dataset to an Azure storage blob storage account.  

In [15]:
!icenet_upload_azure results/predict/example_south_forecast.nc 2020-04-30

[01-02-24 08:44:00 :INFO    ] - Azure upload facility
[01-02-24 08:44:00 :ERROR   ] - Configuration is not correctly set up
Traceback (most recent call last):
  File "/data/hpcdata/users/matsco/conda-envs/icenet0.2.7/lib/python3.8/site-packages/icenet/process/azure.py", line 50, in upload
    url = ini.get("azure", "connection_string")
  File "/data/hpcdata/users/matsco/conda-envs/icenet0.2.7/lib/python3.8/configparser.py", line 781, in get
    d = self._unify_values(section, vars)
  File "/data/hpcdata/users/matsco/conda-envs/icenet0.2.7/lib/python3.8/configparser.py", line 1149, in _unify_values
    raise NoSectionError(section) from None
configparser.NoSectionError: No section: 'azure'
Traceback (most recent call last):
  File "/data/hpcdata/users/matsco/conda-envs/icenet0.2.7/bin/icenet_upload_azure", line 8, in <module>
    sys.exit(upload())
  File "/data/hpcdata/users/matsco/conda-envs/icenet0.2.7/lib/python3.8/site-packages/icenet/process/azure.py", line 53, in upload
    raise

## Other Pipeline Considerations

### A bit more information on ensemble runs

#### Cleaning up runs

Ensemble runs take place under `/ensemble/` in the pipeline folder and ARE NOT deleted after they've happened, to allow for debugging. Commonly, the ensemble configurations will contain a delete task to remove the extraneous run folders. __In the meantime this should be done manually__ after running `run_train_ensemble` or `run_predict_ensemble`.

The only exception to this is the use of `run_daily.sh` (see below) which does clean up prior to rerunning. 

### Daily execution

Daily execution is facilitated in the pipeline by using [`run_daily.sh`](https://github.com/antarctica/IceNet-Pipeline/blob/main/run_daily.sh). This wraps all the necessary steps to perform the following sequence for producing forecasts from yesterday for the next 93 days, for both northern and southern hemispheres. 

* Removes any old ensemble runs
* Downloads [HRES forecast data from the ECMWF MARS API](https://www.ecmwf.int/en/forecasts/datasets/catalogue-ecmwf-real-time-products)
* Processes the HRES and necessary training metadata to produce a data loader
* Creates a dataset configuration for it
* Runs a [prediction ensemble](#Predict) to produce a NetCDF
* Uploads to the necessary endpoint

#### Automation

With the above shell script it's trivial to automate using cron. Of course this is simply for demonstration, with more complex workflow managers offering far great flexibility especially when considering analysis of the produced forecasts.

```bash
# We assume your environment is configured appropriately to run conda from cron files, for example by adding...
#
# SHELL=/bin/bash
# BASH_ENV=~/.bashrc_env
#
# With conda initialisation in bashrc_env at the top of your crontab
25 9 * * * conda activate icenet; cd $HOME/hpc/icenet/pipeline && bash run_daily.sh >$HOME/daily.log 2>&1; conda deactivate
```

TODO: more information on the usage of this command.

## Summary

Within this notebook we've attempted to give a full crash course to running the CLI tools both __manually__ and using the __pipeline helper scripts__. This is the first of four (currently) notebooks contained within the pipeline repository, covering further information: 

* [Data structure and analysis](02.data_analysis.ipynb): understand the structure of the data stores and products created by these workflows and what tools currently exist in IceNet to looks over them.
* [Library usage](03.library_usage.ipynb): understand how to programmatically perform an end to end run.
* [Library extension](04.library_extension.ipynb): understand why and how to extend the IceNet library.

## Version
- IceNet Codebase: v0.2.5